## Welcome to your notebook.


In [ ]:
# Import required libraries/modules
import requests
import ipywidgets as widgets
from arcgis.gis import GIS

In [ ]:
# Create connection and generate token for use in REST Api
gis = GIS('Home')
token = gis._con.token

In [ ]:
# GROUP CHOOSER
# Let user pick a group, then use that group to filter layer lists below

In [ ]:
# Search your content for Feature Layers (including Feature Layer Views) to select for updating and sourcing
# *** To do: Pull from GROUP instead of OWNER *** TRY TO FILTER OUT VIEW LAYER in View Layer List (ex. Search for "Created from: RegionalDistrict_20210603, Feature Layer")
item_search = {item.title:item.id for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, item_type="Feature Layer", max_items=500)}
FeatureLayers_list = sorted([x for x in item_search.keys()])
FeatureLayers_list.insert(0, "")

ViewLayer_Box = widgets.Dropdown(placeholder='Choose Item Type',options=FeatureLayers_list,value=None,description='View Layer',ensure_option=True)
SourceLayer_Box = widgets.Dropdown(placeholder='Choose Item Type',options=FeatureLayers_list,value=None,description='Source Layer',ensure_option=True)
SearchBox = widgets.VBox([ViewLayer_Box,SourceLayer_Box])
display(SearchBox)

In [ ]:
# Variables for which View to update, and which source feature layer to update it with, if blank message to no longer proceed will print
if not ViewLayer_Box.value:
    print("No View Layer Selected, Do Not Continue")
else:
    ViewLayer_Name = ViewLayer_Box.value
    print(ViewLayer_Name)
    
if not SourceLayer_Box.value:
    print("No Source Layer Selected, Do Not Continue")
else:
    SourceLayer_Name = SourceLayer_Box.value
    print(SourceLayer_Name)

In [ ]:
# Parameters to perform REST function (Delete From Definition), request must be POST
Delete_Url = r"https://services6.arcgis.com/ubm4tcTYICKBpist/arcgis/rest/admin/services/%s/FeatureServer/deleteFromDefinition?token=" % (ViewLayer_Name) + str(token)
Delete_params = {'deleteFromDefinition':'{"layers": [{"id":0}]}'}

In [ ]:
# Run POST request to delete from service defintion and check status, 200 = good
Delete_response = requests.post(Delete_Url,params=Delete_params)
Delete_status = Delete_response.status_code
print(Delete_status)

In [ ]:
# Parameters to perform REST function (Add to Definition), request must be POST
SourceLayer_ID = item_search[SourceLayer_Name]
print(SourceLayer_ID)

Add_Url = r"https://services6.arcgis.com/ubm4tcTYICKBpist/arcgis/rest/admin/services/%s/FeatureServer/addToDefinition?token=" % (ViewLayer_Name) + str(token)
Add_params = {'addToDefinition':"""{
                                "layers" : [
                                    {
                                      "adminLayerInfo" : {
                                        "geometryField" : {"name" : "Shape"}, 
                                        "xssTrustedFields" : "", 
                                        "viewLayerDefinition" : {
                                          "sourceServiceName" : "%s", 
                                          "sourceLayerId" : 0, 
                                          "sourceLayerFields" : "*"
                                        }
                                      }, 
                                      "id" : 0, 
                                      "name" : "%s", 
                                      "type" : "Feature Layer", 
                                      "serviceItemID": "%s",
                                      "isView" : true, 
                                      "isUpdatableView" : true, 
                                      "sourceSchemaChangesAllowed" : true, 
                                      "displayField" : "", 
                                      "description" : "", 
                                      "copyrightText" : "", 
                                      "defaultVisibility" : true
                                      }]
                                }
                                """ % (SourceLayer_Name,SourceLayer_Name,SourceLayer_ID)}
                        

In [ ]:
# Run POST request to add to service defintion and check status, 200 = good
Add_response = requests.post(Add_Url,params=Add_params)
Add_status = Add_response.status_code
print(Add_status)

In [ ]:
# ***IN PROGRESS***
# Optional, reapply layer properties that were previously set on View Layer
Update_URL = r"https://services6.arcgis.com/ubm4tcTYICKBpist/arcgis/rest/admin/services/%s/FeatureServer/0?token=" % (ViewLayer_Name) + str(token)